In [1]:
import os
import re
import time 
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=MY_ENV_VAR)
print("client ready")

client ready


In [23]:
System_prompt = """ 
Given the user's goal, generate a solution report. 
---
TAWs to be specified as thoughts(T)+action(A)+work(W):
Set of action that can be performed - (search), (retrival), (analysis)
Example: - {"T": "what all companies are registered and operating in Delhi", "A": "(search)Companies in Delhi","W":"Make a list o names of 40-50 companies in Delhi"}

Template given below (in markdown format):

---
# Solution Report

## Problem Statement
{Describe the user's problem statement}

## Solution
{Describe the solution to the user's problem and how it solves the problem by taking someone's help}

## Target
{List of people need to target who can solve or help user to solve the problem}

## Chain of TAWs
{Give a list of multiple TAWs that can be used to solve the problem}
"""

In [24]:
Goal = "I want to sell my electrical device to all meduim scale industries in Delhi"

In [25]:

start_time = time.time()

response = client.chat.completions.create(
  model= "gpt-3.5-turbo",
  # response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": System_prompt},
    # *question_merge_few_shot,
    {"role": "user", "content": f"Goal:{Goal}"},
  ]
)

end_time = time.time()
print(response)

print(f'Total Tokens:{response.usage.total_tokens}, Prompt Token {response.usage.prompt_tokens}, Completion Token {response.usage.completion_tokens}')
print(f'Time Taken: {end_time - start_time} Sec\n')

# response = json.loads(response.choices[0].message.content)
print(f"Goal: {Goal}")
print(f"Solution: {response.choices[0].message.content}")


ChatCompletion(id='chatcmpl-92CDLi04agfuVjrZcZk3tyJBOhqEX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='# Solution Report\n\n## Problem Statement\nThe user wants to sell their electrical device to all medium-scale industries in Delhi.\n\n## Solution\nTo reach out and sell the electrical device to medium-scale industries in Delhi, the user can start by creating a list of potential companies to target. This can be achieved by conducting market research to identify medium-scale industries in Delhi that would be interested in buying the electrical device. Once the list is prepared, the user can then reach out to these companies through various channels such as cold calls, emails, or in-person meetings to pitch the product and negotiate deals.\n\n## Target\n- Marketing professionals who can help in identifying potential companies.\n- Sales professionals who can assist in pitching the product to target companies.\n\n## Chain of TAWs\n1

Solution: 
# Solution Report

## Problem Statement
The user wants to create a website for their business.

## Solution
To create a website for your business, you can follow these steps:
1. Define the purpose and goals of your website.
2. Choose a domain name and register it with a domain registrar.
3. Select a web hosting service to store your website files.
4. Design your website using a website builder, content management system (CMS) like WordPress, or hire a web developer.
5. Create content for your website, including text, images, and multimedia.
6. Optimize your website for search engines (SEO) to improve visibility.
7. Test your website for functionality and responsiveness on different devices.
8. Launch your website and promote it through various marketing channels.

## Target
1. Web developers/designers: They can help you create a professional and visually appealing website.
2. Content creators: They can assist in crafting engaging content for your website.
3. SEO specialists: They can optimize your website for better search engine rankings.
4. Digital marketers: They can help in promoting your website and driving traffic.

## Chain of work
1. Research and identify web developers/designers who have experience in creating websites for businesses.
2. Discuss your requirements and budget with the selected web developers/designers to get quotes.
3. Collaborate with content creators to develop compelling content for your website.
4. Consult with SEO specialists to optimize your website for search engines.
5. Coordinate with digital marketers to create a marketing strategy for your website launch.

In [ ]:
{
  "name": "search_google",
  "description": "Gives list of website, from google search. Also other details like title, description, url, etc.",
  "parameters": {
    "type": "object",
    "properties": {
      "search_query": {
        "type": "string",
        "description": "Google search string"
      },
      "country_code":{
        "type": "string",
        "description": "Country code for google search eg:IN, US, UK"
      }
    },
    "required": [
      "search_query"
    ]
  }
}


{
  "name": "scrape_website",
  "description": "Take list of website URLs and scrape the website and return the data",
  "parameters": {
    "type": "object",
    "properties": {
      "links": {
        "type": "List[dict]",
        "description": "Takes list of documents dictionary. Each dictionary should have url, title, description"
      },
      "country_code":{
        "type": "string",
        "description": "Country code for google search eg:IN, US, UK"
      }
    },
    "required": [
      "search_query"
    ]
  }
}

In [ ]:
def Stream_agent(prompt):
    messages = [
        { "role": "system", "content": System_prompt },
        { "role": "user", "content": prompt },
    ]
    def extract_action_and_input(text):
          action_pattern = r"Action: (.+?)\n"
          input_pattern = r"Action Input: \"(.+?)\""
          action = re.findall(action_pattern, text)
          action_input = re.findall(input_pattern, text)
          return action, action_input
    while True:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            top_p=1,)
        response_text = response.choices[0].message.content
        print(response_text)
        #To prevent the Rate Limit error for free-tier users, we need to decrease the number of requests/minute.
        time.sleep(20)
        action, action_input = extract_action_and_input(response_text)
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "Response To Human":
            print(f"Response: {action_input[-1]}")
            break
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

"During this morning's meeting, all world conflicts were resolved."

In [ ]:
url = "https://biology.ucdavis.edu/faculty/molecular-and-cellular-biology"

In [8]:
from langchain_community.llms import Ollama

In [9]:
llm = Ollama(model="mistral")

In [12]:
query = "I want an internship in UC Davis in the field of biology, what should I search on google?"

In [13]:
for chunks in llm.stream(query):
    print(chunks, end="")

 To find information about biology internships at the University of California, Davis, you can try using the following search queries on Google:

1. "UC Davis biology internships"
2. "Biology research internships UC Davis"
3. "UC Davis undergraduate biology internships"
4. "UC Davis graduate biology internships"
5. "UC Davis biology lab internships"
6. "Biology industry internships UC Davis"
7. "UC Davis biology research opportunities"
8. "UC Davis biology student internships"

Make sure to explore the websites of the specific departments or programs at UC Davis that align with your area of interest in biology. Additionally, check out the career services center and student organizations related to biology for potential opportunities.

In [7]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://keralaroadlines.com/kochi-packers-movers-kerala","https://threebestrated.in/packers-and-movers-in-kochi-kl","https://www.santaclaracounty.gov/home","https://www.sofitellegendsantaclara.com/","https://lilianweng.github.io/posts/2023-06-23-agent/", "https://santaclaraconventioncenter.com/catering","https://python.langchain.com/docs/use_cases/web_scraping", "https://leverageedu.com/blog/can-we-go-for-phd-after-bachelors/", "https://towardsdatascience.com/machine-learning-basics-part-1-a36d38c7916"]

print("length of urls", len(urls))
loader = AsyncHtmlLoader(urls)
docs = loader.load()

length of urls 10


Fetching pages: 100%|##########| 10/10 [00:09<00:00,  1.11it/s]


In [6]:
schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}

[Document(page_content='\n\t<!DOCTYPE html>\n\t<html class="no-icon-fonts" lang="en">\n\t<head>\n\t\t<meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n<meta http-equiv="x-ua-compatible" content="IE=edge,chrome=1" />\n<meta name="viewport" content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n<meta name="referrer" content="origin-when-cross-origin">\n<link rel="canonical" href="https://www.espn.com" />\n<title>ESPN - Serving Sports Fans. Anytime. Anywhere.</title>\n<meta name="description" content="Visit ESPN for live scores, highlights and sports news. Stream exclusive games on ESPN+ and play fantasy sports." />\n<link rel="manifest" href="/manifest.json">\n<meta property="fb:app_id" content="116656161708917" />\n<meta property="og:site_name" content="ESPN.com" />\n<meta property="og:url" content="https://www.espn.com" />\n<meta property="og:title" content="ESPN - Serving Sports Fans. Anytime. Anywhere."/>\n<meta property="og:description" content="Visi